In [1]:
!cp -r "../input/slices/T1_slices/T1_slices" "/kaggle/working"
!cp -r "../input/slices/T2_slices/T2_slices"  "/kaggle/working"

In [2]:
!cp  "../input/updatedcheckpointssdp/discT1.pth.tar"  "/kaggle/working"
!cp  "../input/updatedcheckpointssdp/discT2.pth.tar"  "/kaggle/working"
!cp  "../input/updatedcheckpointssdp/genT1.pth.tar"  "/kaggle/working"
!cp  "../input/updatedcheckpointssdp/genT2.pth.tar"  "/kaggle/working"

### Importing libraries,packages and modules

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image
import os
from torch.utils.data import Dataset
import numpy as np
import nibabel as nib
import nibabel as nib
import skimage.transform as skTrans
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import os
from torch.utils.data import Dataset
import numpy as np



### Writing custom-dataset class and creating the instance for letting the python access our dataset

In [4]:
class contrastNoncontrastDataset():
    def __init__(self, root_T1, root_T2, transform=None):
        self.root_T1 = root_T1
        self.root_T2 = root_T2
        self.transform = transform

        self.T1_images = os.listdir(root_T1)
        self.T2_images = os.listdir(root_T2)
        self.length_dataset = max(len(self.T1_images), len(self.T2_images)) 
        self.T1_len = len(self.T1_images)
        self.T2_len = len(self.T2_images)

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        T1_img = self.T1_images[index % self.T1_len]
        T2_img = self.T2_images[index % self.T2_len]

        T1_path = os.path.join(self.root_T1, T1_img)
        T2_path = os.path.join(self.root_T2, T2_img)
        T1_img = Image.open(T1_path)
        T2_img = Image.open(T2_path)
        T1_img = T1_img.resize((200,200))
        T2_img = T2_img.resize((200,200))
        if self.transform:
            T1_img = self.transform(T1_img)
            T2_img = self.transform(T2_img)
        

        return T1_img.float()[1].view(1,200,200),T2_img.float()[1].view(1,200,200)

### Class for conv and residual blocks 

In [5]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.block(x)

### Building the generator class by creating instances of conv and residual blocks

In [6]:
class Generator(nn.Module):
    def __init__(self, img_channels=1, num_features = 64, num_residuals=9):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace=True),
        )
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(num_features, num_features*2, kernel_size=3, stride=2, padding=1),
                ConvBlock(num_features*2, num_features*4, kernel_size=3, stride=2, padding=1),
            ]
        )
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4) for _ in range(num_residuals)]
        )
        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(num_features*4, num_features*2, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
                ConvBlock(num_features*2, num_features*1, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
            ]
        )

        self.last = nn.Conv2d(num_features*1, img_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect")

    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.res_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))

### Creating conv block and building the discriminator by creating instances

In [7]:
import torch
import torch.nn as nn

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, 1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=1, features=[64,128,256,512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(Block(in_channels, feature, stride=1 if feature==features[-1] else 2))
            in_channels = feature
        layers.append(nn.Conv2d(in_channels, 1, kernel_size=7, stride=2, padding=1, padding_mode="reflect"))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))
    


### Defining device, path to dataset, batch size, no of epochs and learning rates for generator and discriminator training and storing them in configuration variable

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
config = {
    "DEVICE":device,
    "GENLR":3e-4,
    "DISCLR":3e-4,
    "EPOCHS":100,
    "ROOT_T1":"./T1_slices/",
    "ROOT_T2":"./T2_slices/",
    "TRANSFORMS":torchvision.transforms.ToTensor(),
    "BATCH_SIZE":5
}

### Functions to save and load checkpoints during training

In [9]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=config["DEVICE"])
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

   
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr
def seed_everything(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [10]:
!mkdir saved_images

### Training function

In [11]:


import numpy as np
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image




def train_fn(discT1, discT2, genT2, genT1, train_loader, opt_disc, opt_gen,L1_Loss,Mse_Loss,curr_epoch):
    t1_reals = 0
    t1_fakes = 0
   
    for idx, (t1, t2) in enumerate(train_loader):
        t2 = t2.to(config["DEVICE"])
        t1 = t1.to(config["DEVICE"])
        idx += 1
        
        ## Training discriminators 
        
        
        ## Training discriminator D(G(T2))      G(T1) --> Generator that takes T2 images as input and generates T1 images
        fake_t1 = genT1(t2)       
        D_t1_real = discT1(t1)    
        D_t1_fake = discT1(fake_t1.detach())
        t1_reals += D_t1_real.mean().item()
        t1_fakes += D_t1_fake.mean().item()
        D_LossT1_real = Mse_Loss(D_t1_real, torch.ones_like(D_t1_real))
        D_LossT1_fake = Mse_Loss(D_t1_fake, torch.zeros_like(D_t1_fake))
        D_LossT1 = D_LossT1_real + D_LossT1_fake
        
        ## Training discriminator D(G(T2))      G(T1) --> Generator that takes T1 images as input and generates T2 images
        fake_t2 = genT2(t1)
        D_t2_real = discT2(t2)
        D_t2_fake = discT2(fake_t2.detach())
        D_LossT2_real = Mse_Loss(D_t2_real, torch.ones_like(D_t2_real))
        D_LossT2_fake = Mse_Loss(D_t2_fake, torch.zeros_like(D_t2_fake))
        D_LossT2 = D_LossT2_real + D_LossT2_fake

        ## Combing loss of both the discriminators
        D_loss = (D_LossT1 + D_LossT2)/2
        
        
        ## running back-prop and updating the parameters
        opt_disc.zero_grad()
        D_loss.backward()
        opt_disc.step()
        

        
       
        ## Training generatos 
            
        D_t1_fake = discT1(fake_t1)     #getting the output of descriminator D(G(T2)) by passing generated T1 images
        D_t2_fake = discT2(fake_t2)     #getting the output of descriminator D(G(T1)) by passing generated T2 images
        loss_G_t1 = Mse_Loss(D_t1_fake, torch.ones_like(D_t1_fake))  #Explicitly declaring the generated samples as real
        loss_G_t2 = Mse_Loss(D_t2_fake, torch.ones_like(D_t2_fake))

           
        #Including cycle consistency in the network
        
        cycle_t2 = genT2(fake_t1)  ## generating T2 image by passing generated T1 image as argument to G(T1)
        cycle_t1 = genT1(fake_t2)  ## generating T1 image by passing generated T2 image as argument to G(T2)
        C_consistency_T2 = L1_Loss(t2, cycle_t2)  #Cycle consistency between t2->t1->t2 and t2
        C_consistency_T1 = L1_Loss(t1, cycle_t1)   #Cycle consistency between  t1->t2->t1 and t1

            
        # Incorporating identity loss in both the generators
        identity_t2 = genT2(t2)
        identity_t1 = genT1(t1)
        identity_t2_loss = L1_Loss(t2, identity_t2)
        identity_t1_loss = L1_Loss(t1, identity_t1)

        #summing up all the losses
        G_loss = (
                loss_G_t2
                + loss_G_t1
                + C_consistency_T2 * 10
                + C_consistency_T1 * 10 
                + identity_t1_loss 
                + identity_t2_loss 
            )
        #Running backprop and updating the paramters
        
        opt_gen.zero_grad()
        G_loss.backward()
        opt_gen.step()
        if idx % 100 == 0:
            #save_image(fake_t1*0.5+0.5, f"saved_images/t1_{idx}.png")
            save_image(fake_t2*0.5+0.5, f"saved_images/t2_{idx}.png")
            print(f" Epoch :  {curr_epoch+1:5}     batch: {idx*5}/{len(train_loader)*5}    T2 :- Generator loss: {loss_G_t2:10.4f}%     Descriminator loss : {D_loss:10.4f}% ")
    return genT1,genT2,discT1,discT2



def train_model(numepochs,checkpoints=False,saved_model_genT1=None,saved_model_genT2=None,saved_model_discT1=None,saved_model_discT2=None):
    discT1 = Discriminator().to(config["DEVICE"])
    discT2 = Discriminator().to(config["DEVICE"])
    genT2  = Generator(img_channels=1, num_residuals=9).to(config["DEVICE"])
    genT1  = Generator(img_channels=1, num_residuals=9).to(config["DEVICE"])
    
    opt_disc = optim.Adam(
        list(discT1.parameters()) + list(discT2.parameters()),
        lr=config["DISCLR"],
        betas=(0.5, 0.999),
    )

    opt_gen = optim.Adam(
        list(genT2.parameters()) + list(genT1.parameters()),
        lr=config["GENLR"],
        betas=(0.5, 0.999),
    )
    if checkpoints:
        print("loading checkpoints")
        
        load_checkpoint(
          "./genT1.pth.tar"  , genT1, opt_gen, config["GENLR"],
        )
        load_checkpoint(
          "./genT2.pth.tar"  , genT2, opt_gen, config["GENLR"],
        )
        load_checkpoint(
             "./discT1.pth.tar"  , discT1, opt_disc, config["DISCLR"],
        )
        load_checkpoint(
             "./discT2.pth.tar"  , discT2, opt_disc, config["DISCLR"],
        )
        

       
    L1_Loss = nn.L1Loss()
    Mse_Loss = nn.MSELoss()

    

    dataset = contrastNoncontrastDataset(
        root_T1 = config["ROOT_T1"],root_T2 =config["ROOT_T2"],transform = config["TRANSFORMS"]
    )
    
   
    train_loader = DataLoader(
        dataset,
        batch_size=config["BATCH_SIZE"],
        shuffle=True,
        
    )
    device = config["DEVICE"]
    print(f"Training the model on {device}") 
    for epoch in tqdm(range(numepochs)):
        genT1,genT2,discT1,discT2 = train_fn(discT1, discT2, genT2, genT1, train_loader, opt_disc, opt_gen,L1_Loss,Mse_Loss,epoch)
        save_checkpoint(genT1, opt_gen, filename="genT1.pth.tar")
        save_checkpoint(genT2, opt_gen, filename="genT2.pth.tar")
        
        save_checkpoint(discT1, opt_disc, filename="discT1.pth.tar")
        save_checkpoint(discT2, opt_disc, filename="discT2.pth.tar")
    return genT1,genT2,discT1,discT2

        
    



### Training!

In [12]:
genT1,genT2,discT1,discT2 = train_model(100,True)

Training the model on cuda


  0%|          | 0/100 [00:00<?, ?it/s]

 Epoch :      1     batch: 500/1060    T2 :- Generator loss:     0.8768%     Descriminator loss :     0.3187% 
 Epoch :      1     batch: 1000/1060    T2 :- Generator loss:     0.9893%     Descriminator loss :     0.3578% 
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


  1%|          | 1/100 [04:01<6:37:51, 241.12s/it]

 Epoch :      2     batch: 500/1060    T2 :- Generator loss:     0.9343%     Descriminator loss :     0.1703% 
 Epoch :      2     batch: 1000/1060    T2 :- Generator loss:     0.4696%     Descriminator loss :     0.7250% 
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


...

 98%|█████████▊| 98/100 [6:24:32<07:50, 235.42s/it]

 Epoch :     99     batch: 500/1060    T2 :- Generator loss:     0.9937%     Descriminator loss :     0.0133% 
 Epoch :     99     batch: 1000/1060    T2 :- Generator loss:     0.9938%     Descriminator loss :     0.0110% 
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


 99%|█████████▉| 99/100 [6:28:27<03:55, 235.40s/it]

 Epoch :    100     batch: 500/1060    T2 :- Generator loss:     0.9999%     Descriminator loss :     0.0533% 
 Epoch :    100     batch: 1000/1060    T2 :- Generator loss:     0.9909%     Descriminator loss :     0.0175% 
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint
=> Saving checkpoint


100%|██████████| 100/100 [6:32:22<00:00, 235.43s/it]
